In [1]:
#het importeren van de benodigheden
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, text
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

## Sprint 2.1:

# Beschrijving van de data

We hebben de tabellen Country, League, Match, Player, Player_Attributes, Team en Team_Attributes

In de Country tabel staan de landen die gebruikt worden met daar bij een land id.

In de League tabel staan de verschillende leagues die gebruikt worden met daarbij het bijbehorende land, ook staat in de tabel een land id en een league id.

In de Match tabel staat de meeste informatie, je hebt hier staan wat de uitslag is, welke positie een speler heeft op het veld via een X en Y grid, Wie spelers waren via speler ID, er zijn een paar kolommen (goal, shoton, shotoff, foulcommit, card, cross, corner, possession) waar informatie staat over de desbetreffende gegevens, die informatie in deze kolommen is heel onduidelijk en we moeten nog kijken hoe we dit gaan toepassen. Ook heb je voor een paar gokwebsites de odds voor de wedstrijd, deze odds zijn van voordat de wedstrijd begon (Home odds, gelijkspel odds en Away odds). Als laatst hebben we een id van de wedstrijd, een lands id, een league id, het seizoen dat het gebeurde, de stage waar het gebeurde (speelweek) en wanneer het gebeurde. Ook hebben we nog een match_api_id, een home_team_api_id en een away_team_api_id.

In de Player tabel staan de verschillende spelers met daarbij hun geboortedatum, lengte en gewicht. Ook hebben ze een ID, player_api_id en een player_fifa_api_id

In de Player_Attributes tabel staan per spelers hun fifa ratings per verschillend onderdeel, denk bijvoorbeeld aan rating, potentie, shooting en dribbling. Ook staan hoeveel moeite ze stoppen in aanvallen en verdedigen in fifa en wat hun favoriete voet is om mee te spelen. Als laatste hebben ze ook nog een ID, een player_fifa_api_id, een player_api_id en de datum dat deze gegevens opgenomen zijn.

In de Team tabel staat een id, een team_api_id, een team_fifa_api_id, een team naam en een afkorting van de team naam.

In de Team_Attributes tabel een id, een team_fifa_api_id, een team_api_id en de datum dat de gegevens afgenomen zijn. Ook staat erin hoe het team speelt in fifa, denk bijvoorbeeld aan hoe wijd ze verdedigen of hoeveel ze hun kansen maken via passen en met wat voor passen ze gebruiken.



In [4]:
#Het laden van de dataframes voor sql
cnx = sqlite3.connect("database.sqlite")

league_df = pd.read_sql_query("SELECT * FROM League", cnx)
match_df = pd.read_sql_query("SELECT * FROM Match", cnx)
team_df = pd.read_sql_query("SELECT * FROM Team", cnx)

Deze 3 cellen hieronder zijn code die we van het vak Data II gekopiëerd hebben zodat we SQL kunnen gebruiken in python

In [ ]:
def show_query(query):
    """
       Deze functie print de output van je SQL query uit.
       Parameters:
                  query = Je SQL query.
                          Zorg ervoor dat deze parameter een string is.
    """
    query_output = engine.execute(query).fetchall()
    column_names = engine.execute(query)._metadata.keys
    return pd.DataFrame(query_output, columns = column_names)

In [ ]:
def df_to_database(df, name):
    """
       Deze functie maakt een database van een dataframe.
       Parameters:
                  df  = de dataframe waarvan je een database tabel wil maken.
                         
                  name = De naam die je de database wil geven.
                         Zorg ervoor dat deze parameter een string is.
    """
    df.to_sql(name, con=engine)
    
    display(df)


In [ ]:
engine = create_engine('sqlite://', echo=False)


In [ ]:
#Hier worden de databases gemaakt zodat we die via sql kunnen gebruiken
df_to_database(team_df, 'team')
df_to_database(league_df, 'league')
df_to_database(match_df, 'match')

In [ ]:
query1 = """

SELECT *
  FROM team
  WHERE team_long_name LIKE '%Fey%'

                  
"""

show_query(query1)

#team id is 167, team_api_id is 10235 en team_fifa_api_id is 246
#Team naam is Feyenoord en afkorting is FEY

In [ ]:
query2 = """

SELECT le.name,le.id
FROM team AS te
INNER JOIN match as ma
ON te.team_api_id = ma.home_team_api_id
INNER JOIN league as le
ON le.country_id = ma.country_id
WHERE te.team_api_id = 10235
LIMIT 1


"""

show_query(query2)

#De league is 'Netherlands Eredivisie' waarvan het id 13274 is

In [ ]:
query3 = """

SELECT ma1.*, te.team_long_name AS Home_team_name, te2.team_long_name AS Away_team_name
FROM match AS ma1
INNER JOIN team as te
ON te.team_api_id = ma1.home_team_api_id
INNER JOIN team AS te2
ON te2.team_api_id = ma1.away_team_api_id
WHERE te.team_api_id = 10235 OR te2.team_api_id = 10235
"""

show_query(query3)
#Je ziet hier alle wedstrijden van feyenoord die in de database staan

In [ ]:
query4 = """

SELECT ma1.*, te.team_long_name AS Home_team_name, te2.team_long_name AS Away_team_name
FROM match AS ma1
INNER JOIN team as te
ON te.team_api_id = ma1.home_team_api_id
INNER JOIN team AS te2
ON te2.team_api_id = ma1.away_team_api_id
WHERE (te.team_api_id = 10235 OR te2.team_api_id = 10235) AND ma1.season= '2008/2009'
GROUP BY Home_team_name

"""

show_query(query4)
#Je ziet hier alle wedstrijden van feyenoord die in de database staan

# Sprint 3.1:

In [ ]:
def matches_played(team):
    """
    Deze functie laat zien hoeveel wedstrijden een zelf gekozen team in elk seizoen gespeelt heeft
    Parameters:
            team (int): het team waarvan je het wil zien
            
        Returns:
            show_query(query5) (pandas dataframe): geeft een dataframe terug met de data
    """
    
    
    query5 = f"""

    SELECT ma1.season, COUNT(ma1.match_api_id) AS matches_played
    FROM match AS ma1
    INNER JOIN team as te
    ON te.team_api_id = ma1.home_team_api_id
    INNER JOIN team AS te2
    ON te2.team_api_id = ma1.away_team_api_id
    WHERE (te.team_api_id = {team} OR te2.team_api_id = {team})
    GROUP BY season


    """

    return show_query(query5)

matches_played(10235)

#je ziet hier dat feyenoord 34 wedstrijden gespeeld heeft elk seizoen.

In [ ]:
def matches_year(team,year):
    """
    Deze functie laat zien hoeveel wedstrijden een zelf gekozen team in een zelf gekozen jaar
    Parameters:
            team (int): het team waarvan je het wil zien
            year (str): het jaar waarvan je het wil zien
            
        Returns:
            show_query(query6) (pandas dataframe): geeft een dataframe terug met de data
    """
    query6 = f"""

    SELECT ma1.season, COUNT(ma1.match_api_id) AS matches_played
    FROM match AS ma1
    INNER JOIN team as te
    ON te.team_api_id = ma1.home_team_api_id
    INNER JOIN team AS te2
    ON te2.team_api_id = ma1.away_team_api_id
    WHERE (te.team_api_id = {team} OR te2.team_api_id = {team}) AND ma1.date LIKE '{year}%'
    GROUP BY season


    """

    return show_query(query6)
matches_year(10235,2010)
#je ziet dat feyenoord in het jaar 2010 in het 2009/2010 seizoen 16 wedstrijden heeft gespeeld en in het 2010/2011 seizoen 18 wedstrijden heeft gespeeld

Ik heb de mega_ranglijst functie hieronder gemaakt door via WITH 2 queries toe te voegen die de home punten en de away punten en de goals hebben waardoor ik uiteindelijk via de hoofdquery het bij elkaar kan tellen en linken op het team en seizoen en dat nog gesorteerd op eerst seizoen en daarna punten en dan ook nog op doelpunten saldo. Ik heb ook nog toegevoegd dat posities geteld word voor elk seizoen.

In [ ]:
def mega_ranglijst(league):
        """
        Deze functie laat alle ranglijsten van een league zien
        Parameters:
                league (int): het league waarvan je het wil zien

            Returns:
                show_query(query7) (pandas dataframe): geeft een dataframe terug met de data
        """
        query7 = f"""
        WITH test4 AS
        (SELECT ma1.season, ma1.home_team_api_id AS home_team, te.team_long_name, SUM(ma1.home_team_goal) AS home_goals_scored, SUM(ma1.away_team_goal) AS home_goals_conceited,
        SUM(CASE WHEN ma1.home_team_goal > ma1.away_team_goal THEN 3
             WHEN ma1.home_team_goal = ma1.away_team_goal THEN 1
             ELSE 0 END) AS home_punten


        FROM match AS ma1

        INNER JOIN team as te
        ON te.team_api_id = ma1.home_team_api_id
        INNER JOIN team AS te2
        ON te2.team_api_id = ma2.away_team_api_id
        INNER JOIN match as ma2
        ON ma1.match_api_id = ma2.match_api_id
        WHERE ma1.league_id = {league} 
        GROUP BY ma1.season, te.team_api_id
        ORDER BY ma1.season),

       
        test3 AS
        (SELECT ma1.season, ma2.away_team_api_id AS away_team, te2.team_long_name, SUM(ma2.away_team_goal) AS away_goals_scored, SUM(ma2.home_team_goal) AS away_goals_conceited,
        SUM(CASE WHEN ma2.home_team_goal < ma2.away_team_goal THEN 3
             WHEN ma2.home_team_goal = ma2.away_team_goal THEN 1
             ELSE 0 END) AS away_punten

        FROM match AS ma1

        INNER JOIN team as te
        ON te.team_api_id = ma1.home_team_api_id
        INNER JOIN team AS te2
        ON te2.team_api_id = ma2.away_team_api_id
        INNER JOIN match as ma2
        ON ma1.match_api_id = ma2.match_api_id
        WHERE ma2.league_id = {league}


        GROUP BY ma2.season, te2.team_api_id
        ORDER BY away_punten) 


        SELECT te4.season,
        RANK() OVER(PARTITION BY te4.season ORDER BY (te4.home_punten + te8.away_punten) DESC, ((te4.home_goals_scored + te8.away_goals_scored)  - (te4.home_goals_conceited + te8.away_goals_conceited)) DESC) AS Position,
        te4.home_team AS Team_id, te4.team_long_name AS Team_name, (te4.home_punten + te8.away_punten) AS totale_punten,
        (te4.home_goals_scored + te8.away_goals_scored)  - (te4.home_goals_conceited + te8.away_goals_conceited)  AS goal_saldo
        
        FROM test4 AS te4
        INNER JOIN test3 AS te8
        ON te4.home_team = te8.away_team AND te4.season = te8.season

        GROUP BY te4.season, te4.home_team

        ORDER BY te4.season, totale_punten DESC, goal_saldo DESC

        """

        return show_query(query7)


mega_ranglijst(13274)
#Je ziet hier alle seizoenen van de Eredivisie

Ik heb de functie hierboven gekopieerd en eigenlijk alleen een extra parameter toegevoegd waarmee je het op een specifiek seizoen kan zoeken.

In [ ]:
def seizoens_ranglijst(league,seizoen):
        """
        Deze functie laat een ranglijst zien van een zelfgekozen league en seizoen
        Parameters:
                league (int): het team waarvan je het wil zien
                seizoen (str): het seizoen waarvan je het wil zien

            Returns:
                show_query(query7) (pandas dataframe): geeft een dataframe terug met de data
        """
        query7 = f"""
        WITH test4 AS
        (SELECT ma1.season, ma1.home_team_api_id AS home_team, te.team_long_name, SUM(ma1.home_team_goal) AS home_goals_scored, SUM(ma1.away_team_goal) AS home_goals_conceited,
        SUM(CASE WHEN ma1.home_team_goal > ma1.away_team_goal THEN 3
             WHEN ma1.home_team_goal = ma1.away_team_goal THEN 1
             ELSE 0 END) AS home_punten


        FROM match AS ma1

        INNER JOIN team as te
        ON te.team_api_id = ma1.home_team_api_id
        INNER JOIN team AS te2
        ON te2.team_api_id = ma2.away_team_api_id
        INNER JOIN match as ma2
        ON ma1.match_api_id = ma2.match_api_id
        WHERE ma1.league_id = {league} 
        GROUP BY ma1.season, te.team_api_id
        ORDER BY ma1.season),

       
        test3 AS
        (SELECT ma1.season, ma2.away_team_api_id AS away_team, te2.team_long_name, SUM(ma2.away_team_goal) AS away_goals_scored, SUM(ma2.home_team_goal) AS away_goals_conceited,
        SUM(CASE WHEN ma2.home_team_goal < ma2.away_team_goal THEN 3
             WHEN ma2.home_team_goal = ma2.away_team_goal THEN 1
             ELSE 0 END) AS away_punten

        FROM match AS ma1

        INNER JOIN team as te
        ON te.team_api_id = ma1.home_team_api_id
        INNER JOIN team AS te2
        ON te2.team_api_id = ma2.away_team_api_id
        INNER JOIN match as ma2
        ON ma1.match_api_id = ma2.match_api_id
        WHERE ma2.league_id = {league}


        GROUP BY ma2.season, te2.team_api_id
        ORDER BY away_punten) 


        SELECT te4.season,RANK() OVER(ORDER BY (te4.home_punten + te8.away_punten) DESC,((te4.home_goals_scored + te8.away_goals_scored)  - (te4.home_goals_conceited + te8.away_goals_conceited)) DESC) AS Position, te4.home_team AS Team_id, te4.team_long_name AS Team_name, (te4.home_punten + te8.away_punten) AS totale_punten,  
        
        (te4.home_goals_scored + te8.away_goals_scored)  - (te4.home_goals_conceited + te8.away_goals_conceited)  AS goal_saldo

        FROM test4 AS te4
        INNER JOIN test3 AS te8
        ON te4.home_team = te8.away_team AND te4.season = te8.season
        WHERE te4.season = '{seizoen}'

        GROUP BY te4.season, te4.home_team

        ORDER BY te4.season, totale_punten DESC, goal_saldo DESC

        """

        return show_query(query7)

seizoens_ranglijst(13274,'2008/2009')
#Je ziet dus dat feyenoord 6de was met 45 punten in het seizoen 2008/2009

Ik heb ook nog een extra functie gemaakt waar je kan zien welke plek jouw team is geeindigd in elk seizoen

In [ ]:
def Posities_team(team):
        """
        Deze functie laat alle ranglijsten van een league zien
        Parameters:
                team (str): het team waarvan je het wil zien

            Returns:
                show_query(query7) (pandas dataframe): geeft een dataframe terug met de data
        """
        team_api_id = team_df.loc[(team_df['team_long_name'] == team)]['team_api_id'].values[0]

        league=  match_df.loc[(match_df['home_team_api_id'] == team_api_id)]['league_id'].values[0]

        query7 = f"""
        WITH test4 AS
        (SELECT ma1.season, ma1.home_team_api_id AS home_team, te.team_long_name, SUM(ma1.home_team_goal) AS home_goals_scored, SUM(ma1.away_team_goal) AS home_goals_conceited,
        SUM(CASE WHEN ma1.home_team_goal > ma1.away_team_goal THEN 3
             WHEN ma1.home_team_goal = ma1.away_team_goal THEN 1
             ELSE 0 END) AS home_punten


        FROM match AS ma1

        INNER JOIN team as te
        ON te.team_api_id = ma1.home_team_api_id
        INNER JOIN team AS te2
        ON te2.team_api_id = ma2.away_team_api_id
        INNER JOIN match as ma2
        ON ma1.match_api_id = ma2.match_api_id
        WHERE ma1.league_id = {league} 
        GROUP BY ma1.season, te.team_api_id
        ORDER BY ma1.season),

       
        test3 AS
        (SELECT ma1.season, ma2.away_team_api_id AS away_team, te2.team_long_name, SUM(ma2.away_team_goal) AS away_goals_scored, SUM(ma2.home_team_goal) AS away_goals_conceited,
        SUM(CASE WHEN ma2.home_team_goal < ma2.away_team_goal THEN 3
             WHEN ma2.home_team_goal = ma2.away_team_goal THEN 1
             ELSE 0 END) AS away_punten

        FROM match AS ma1

        INNER JOIN team as te
        ON te.team_api_id = ma1.home_team_api_id
        INNER JOIN team AS te2
        ON te2.team_api_id = ma2.away_team_api_id
        INNER JOIN match as ma2
        ON ma1.match_api_id = ma2.match_api_id
        


        GROUP BY ma2.season, te2.team_api_id
        ORDER BY away_punten),


        test5 AS
        (SELECT te4.season,
        RANK() OVER(PARTITION BY te4.season ORDER BY (te4.home_punten + te8.away_punten) DESC, ((te4.home_goals_scored + te8.away_goals_scored)  - (te4.home_goals_conceited + te8.away_goals_conceited)) DESC) AS Position,
        te4.home_team AS Team_id, te4.team_long_name AS Team_name, (te4.home_punten + te8.away_punten) AS totale_punten,
        (te4.home_goals_scored + te8.away_goals_scored)  - (te4.home_goals_conceited + te8.away_goals_conceited)  AS goal_saldo
        
        FROM test4 AS te4
        INNER JOIN test3 AS te8
        ON te4.home_team = te8.away_team AND te4.season = te8.season

        GROUP BY te4.season, te4.home_team

        ORDER BY te4.season, totale_punten DESC, goal_saldo DESC)
        SELECT te5.season, te5.Team_name, te5.Position
        FROM test5 AS te5
        WHERE te5.Team_name = '{team}'

        """

        return show_query(query7)


Posities_team('Feyenoord')
#Je ziet hier waar Feyenoord is geeindigd in elk seizoen

# Sprint 2.2:

Wij hebben gekozen om met classes te werken omdat dat ons het handigste leek in verband met de opdracht

In [ ]:
class FootballData:
    """
    Class to get and describe the data from kaggle soccer dataset
    """
    def __init__(self, data_path):
        """
        Initialize data from kaggle to use in other methods
        """
        self.cnx = sqlite3.connect(data_path)

        self.country_df = pd.read_sql_query("SELECT * FROM Country", self.cnx)
        self.league_df = pd.read_sql_query("SELECT * FROM League", self.cnx)
        self.match_df = pd.read_sql_query("SELECT * FROM Match", self.cnx)
        self.player_df = pd.read_sql_query("SELECT * FROM Player", self.cnx)
        self.player_att_df = pd.read_sql_query("SELECT * FROM Player_Attributes", self.cnx)
        self.team_df = pd.read_sql_query("SELECT * FROM Team", self.cnx)
        self.team_att_df = pd.read_sql_query("SELECT * FROM Team_Attributes", self.cnx)
        
    def get_league_season_data(self, league, season):
        """
        Return all match data from a competition in a season
        """
        # Get the data from the dataframe

        league_api_id = self.league_df.loc[(self.league_df['name'] == league)]['id'].values[0]
        
        league_matches = self.match_df.loc[(self.match_df['league_id'] == league_api_id) &  
                                           (self.match_df['season'] == season)].copy()
        # Add Home and away column for points to able to differentiate between games played home and games played away

        
        league_matches["points_for_home"] = np.where(league_matches['home_team_goal'] > league_matches['away_team_goal'], 3,
                                             np.where(league_matches['home_team_goal'] == league_matches['away_team_goal'], 1, 0))
        league_matches["points_for_away"] = np.where(league_matches['home_team_goal'] < league_matches['away_team_goal'], 3,
                                             np.where(league_matches['home_team_goal'] == league_matches['away_team_goal'], 1, 0))
        # Split the the dataframe in to home_games and away_games so we can like it to the correct ID's
        
        home_games = league_matches.groupby('home_team_api_id').sum().sort_values("points_for_home")
        away_games = league_matches.groupby('away_team_api_id').sum().sort_values("points_for_away")
        
        # Making our eventual 'Ranglijst' dataframe
        ranglijst = pd.DataFrame()
        
        # Adding the columns to the 'Ranglijst' dataframe
        ranglijst['Team_id']= self.match_df.loc[(self.match_df['league_id'] == league_api_id) &  (self.match_df['season'] == season), 'home_team_api_id'].unique()
        ranglijst['Team_Name'] = pd.merge(ranglijst, self.team_df, left_on =ranglijst['Team_id'], right_on = self.team_df['team_api_id'])['team_long_name']
        ranglijst['Doelpunten_voor'] = pd.merge(ranglijst, home_games, left_on =ranglijst['Team_id'], right_index=True)['home_team_goal'] + pd.merge(ranglijst, away_games, left_on =ranglijst['Team_id'], right_index=True)['away_team_goal']
        ranglijst['Doelpunten_tegen'] = pd.merge(ranglijst, home_games, left_on =ranglijst['Team_id'], right_index=True)['away_team_goal'] + pd.merge(ranglijst, away_games, left_on =ranglijst['Team_id'], right_index=True)['home_team_goal']
        ranglijst['Doelpunten_saldo'] = ranglijst['Doelpunten_voor'] - ranglijst['Doelpunten_tegen']
        ranglijst['Totale_punten'] = pd.merge(ranglijst, home_games, left_on =ranglijst['Team_id'], right_index=True)['points_for_home'] + pd.merge(ranglijst, away_games, left_on =ranglijst['Team_id'], right_index=True)['points_for_away']
        
        # Sorting the 'Ranglijst' dataframe and changing it's index (to make it look more like an actual ranking)
        ranglijst = ranglijst.sort_values(by=['Totale_punten', 'Doelpunten_saldo'], ascending=False, ignore_index=True)
        ranglijst.index= ranglijst.index +1


        return ranglijst
        
    
    def get_team_season_data(self, team, season):
        """
        Return all match data from a team in a particular season
        """
        # Get the data from the dataframe
        team_api_id = self.team_df.loc[(self.team_df['team_long_name'] == team)]['team_api_id'].values[0]
        team_matches = self.match_df.loc[((self.match_df['home_team_api_id'] == team_api_id) | 
                                          (self.match_df['away_team_api_id'] == team_api_id))
                                         & (self.match_df['season'] == season)].copy()
        
        # Add Home and away column to able to differentiate between games played home and games played away
        team_matches["home/away"] = np.where(team_matches['home_team_api_id'] == team_api_id, "home", "away")

        team_matches["punten"] = np.where(((team_matches['home_team_goal'] > team_matches['away_team_goal']) & (team_matches["home/away"] == "home")) |
                                          ((team_matches['home_team_goal'] < team_matches['away_team_goal']) & (team_matches["home/away"] == "away")), 3, 
                                          np.where(team_matches['home_team_goal'] == team_matches['away_team_goal'], 1, 0))
        
        return team_matches
    
        
        
        
        

In [ ]:
#Dit is ook al Sprint 3.2
class Analyse:
    def __init__(self, data, ranglijst):
        self.data = data
        self.ranglijst = ranglijst
        
    def get_att_corr(self, team_att_df, start_date_season, end_date_season):
        team_att_df["date"] = pd.to_datetime(team_att_df["date"])
        
        value_replacements = [[1, 0, 2], 
                              [0, 1, 2], 
                              [1, 0, 2], 
                              [0, 1], 
                              [1, 2, 0], 
                              [0, 2, 1], 
                              [1, 2, 0], 
                              [0, 1],
                              [0, 1, 2], 
                              [0, 1, 2], 
                              [1, 0, 2], 
                              [0, 1]]
        
        for value, cat_col in enumerate(team_att_df.filter(regex="Class$", axis=1)):
            new_col = cat_col + "Nr"
            team_att_df[new_col] = team_att_df[cat_col].replace(team_att_df[cat_col].dropna().unique(), value_replacements[value])
        
        team_att_df = team_att_df.loc[(team_att_df["date"] > start_date_season) & 
                                      (team_att_df["date"] <= end_date_season)].copy()
        analisis_df = self.ranglijst.merge(team_att_df, how='inner', left_on='Team_id', right_on='team_api_id')
        correlations = {}
        
        for att in analisis_df.select_dtypes(include=np.number).columns\
        .difference(["id", "Team_id", "team_fifa_api_id", "team_api_id", "date", 
                     "Totale_punten", "Doelpunten_saldo", "Doelpunten_voor", "Doelpunten_tegen"]):
            if len(analisis_df[att].dropna().unique()) != 1:
                correlations[att] = analisis_df["Totale_punten"].corr(analisis_df[att])
        
        
        correlations_df = pd.DataFrame(correlations.items(), columns=['Labels', 'Values']).set_index('Labels').sort_values('Values', ascending=False)
        fig = plt.figure(figsize=(5, 20))
        fig = sns.heatmap(correlations_df, annot=True)
        return (correlations_df, analisis_df)



        
    def describe_team_season_data(self):
        """
        Print a description of the team/season self.data
        """  
        home_win_perc = round(len(self.data["punten"].loc[(self.data["punten"] == 3) & (self.data["home/away"] == "home")]) / len(self.data.loc[self.data["home/away"] == "home"]) * 100, 2)
        home_draw_perc = round(len(self.data["punten"].loc[(self.data["punten"] == 1) & (self.data["home/away"] == "home")]) / len(self.data.loc[self.data["home/away"] == "home"]) * 100, 2)
        home_loss_perc = round(len(self.data["punten"].loc[(self.data["punten"] == 0) & (self.data["home/away"] == "home")]) / len(self.data.loc[self.data["home/away"] == "home"]) * 100, 2)
        
        away_win_perc = round(len(self.data["punten"].loc[(self.data["punten"] == 3) & (self.data["home/away"] == "away")]) / len(self.data["punten"].loc[self.data["home/away"] == "away"]) * 100, 2)
        away_draw_perc = round(len(self.data["punten"].loc[(self.data["punten"] == 1) & (self.data["home/away"] == "away")]) / len(self.data["punten"].loc[self.data["home/away"] == "away"]) * 100, 2)
        away_loss_perc = round(len(self.data["punten"].loc[(self.data["punten"] == 0) & (self.data["home/away"] == "away")]) / len(self.data["punten"].loc[self.data["home/away"] == "away"]) * 100, 2)
        
        
        print(f"Aantal wedstrijden: {str(len(self.data))} \n\
                Aantal punten: {str(self.data['punten'].sum())} \n\
                Gemiddelde punten per wedstrijd: {str(round(self.data['punten'].mean(), 2))} \n\n\
                Win thuis percentage: {str(home_win_perc)}% \n\
                Gelijk thuis percentage: {str(home_draw_perc)}% \n\
                Verlies thuis percentage: {str(home_loss_perc)}% \n\n\
                Win uit percentage: {str(away_win_perc)}% \n\
                Gelijk uit percentage: {str(away_draw_perc)}% \n\
                Verlies uit percentage: {str(away_loss_perc)}%")
        

In [ ]:
# Initialiseer objecten
fd = FootballData("database.sqlite")
analyse = Analyse(fd.get_team_season_data("Feyenoord", "2013/2014"), 
                  fd.get_league_season_data('Netherlands Eredivisie','2013/2014'))
#het weergeven van de benodigde informatie van ons team in het desbetreffende seizoen
analyse.describe_team_season_data()
#het weergeven van de ranglijst van ons desbetreffende seizoen en competitie
fd.get_league_season_data('Netherlands Eredivisie','2013/2014')

# Sprint 3.2:

In [ ]:
df1, df2 = analyse.get_att_corr(fd.team_att_df, datetime.strptime("2013-01-01", "%Y-%m-%d"), datetime.strptime("2015-01-01", "%Y-%m-%d"))

In [ ]:
def top_5_heatmap_visualised(heatmap_df,df_attributes):
    """
    visualise the top 5 heatmap variables in the appropriate graphs
    """
    #change the index collumn to a normal one
    df6 = heatmap_df.reset_index()
    heatmap_variable=   df6['Labels']
    
    #make the appropriate amount of ghraps
    fig, axes = plt.subplots(3, 2, figsize=(20, 20))
    fig.suptitle("Positive Correlation")
    
    #checks if the amount of unique values is 2 or less to make a countplot if not it makes a regplot
    if len(set(df_attributes[heatmap_variable[0]])) <= 2 :
        sns.countplot(ax=axes[0,0],y=heatmap_variable[0],data=df_attributes,color = "pink",edgecolor="Black").set(ylabel=heatmap_variable[0],yticklabels=(['No', 'Yes']))
    else:
        sns.regplot(ax=axes[0,0],data=df_attributes,y = heatmap_variable[0],x = "Totale_punten",color = "purple",line_kws={"color":"red","linestyle":"-","lw":3},scatter_kws={"edgecolor":"white","alpha":0.5}).set(ylabel= heatmap_variable[0],xlabel='totale punten')
    if len(set(df_attributes[heatmap_variable[1]])) <= 2 :
        sns.countplot(ax=axes[0,1],y=heatmap_variable[1],data=df_attributes,color = "pink",edgecolor="Black").set(ylabel= heatmap_variable[1],yticklabels=(['No', 'Yes']))
    else:
        sns.regplot(ax=axes[0,1],data=df_attributes,y = heatmap_variable[1],x = "Totale_punten",color = "purple",line_kws={"color":"red","linestyle":"-","lw":3},scatter_kws={"edgecolor":"white","alpha":0.5}).set(ylabel= heatmap_variable[1],xlabel='totale punten')
    if len(set(df_attributes[heatmap_variable[2]])) <= 2 :
        sns.countplot(ax=axes[1,0],y=heatmap_variable[2],data=df_attributes,color = "pink",edgecolor="Black").set(ylabel=heatmap_variable[2],yticklabels=(['No', 'Yes']))
    else:
        sns.regplot(ax=axes[1,0],data=df_attributes,y = heatmap_variable[2],x = "Totale_punten",color = "purple",line_kws={"color":"red","linestyle":"-","lw":3},scatter_kws={"edgecolor":"white","alpha":0.5}).set(ylabel= heatmap_variable[2],xlabel='totale punten')
    if len(set(df_attributes[heatmap_variable[3]])) <= 2 :
        sns.countplot(ax=axes[1,1],y=heatmap_variable[3],data=df_attributes,color = "pink",edgecolor="Black").set(ylabel=heatmap_variable[3],yticklabels=(['No', 'Yes']))
    else:
        sns.regplot(ax=axes[1,1],data=df_attributes,y = heatmap_variable[3],x = "Totale_punten",color = "purple",line_kws={"color":"red","linestyle":"-","lw":3},scatter_kws={"edgecolor":"white","alpha":0.5}).set(ylabel= heatmap_variable[3],xlabel='totale punten')
    if len(set(df_attributes[heatmap_variable[4]])) <= 2 :
        sns.countplot(ax=axes[2,0],y=heatmap_variable[4],data=df_attributes,color = "pink",edgecolor="Black").set(ylabel=heatmap_variable[4],yticklabels=(['No', 'Yes']))
    else:
        sns.regplot(ax=axes[2,0],data=df_attributes,y = heatmap_variable[4],x = "Totale_punten",color = "purple",line_kws={"color":"red","linestyle":"-","lw":3},scatter_kws={"edgecolor":"white","alpha":0.5}).set(ylabel= heatmap_variable[4],xlabel='totale punten')

        plt.show()

In [ ]:
top_5_heatmap_visualised(df1,df2)

# Gemiddelde statistieken per speler

Hier worden de fifa statistieken per speler samengevoegd en het gemiddelde ervan berekend

In [ ]:
player_att_df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)

In [ ]:
Average_att = player_att_df.groupby(['player_api_id']).mean()
column_names = ['crossing','finishing','heading_accuracy','short_passing','volleys','dribbling','curve', 'free_kick_accuracy', 'long_passing',
       'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions',
       'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving',
       'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes', 'potential']
len(column_names)

In [ ]:
Average_att['Total']= Average_att[column_names].sum(axis=1)
Average_att['Total2'] = Average_att['Total']/len(column_names)

In [ ]:
Average_att = Average_att["Total2"]
Average_att

In [ ]:
match_df = pd.read_sql_query("SELECT * FROM Match", cnx)

In [ ]:
display(match_df.filter(like="player"))

# rating toevoegen aan match dataframe

Hier word de rating uit de player attributes toegevoegd aan de match dataframe

In [ ]:
def match_with_ratings(rating):
    players = player_att_df.groupby(['player_api_id']).mean().round(0).reset_index()
    player_rating_dict = dict(zip(players["player_api_id"], players[rating]))
    columns = ["home_player_" + str(i) for i in range(1, 12)] + ["away_player_" + str(i) for i in range(1, 12)]
    
    for col in columns:
        col_name = col + "_rating"
        match_df[col_name] = [player_rating_dict.get(i, None) for i in match_df[col]]
    return match_df
display(match_with_ratings().filter(like="player"))

In [5]:
def teamseason(season):
    season = match_df.loc[(match_df['season'] == season)]
    season_team_list = season["home_team_api_id"].unique().tolist()
    
    visualise_df=pd.DataFrame()
    visualise_df['team_api_id'] = season_team_list
    gem_punten = []
    for team_id in season_team_list:
        # run your code for each value in the list
        result10 = match_df.loc[(match_df['away_team_api_id'] == team_id)& (match_df['season'] == season)  | (match_df['home_team_api_id'] == team_id) & (match_df['season'] == season)]
    
        home = result10.loc[(match_df['home_team_api_id'] == team_id)& (result10['season'] == season)].copy()
        away= result10.loc[(match_df['away_team_api_id'] == team_id)& (result10['season'] == season)].copy()
        home['punten'] = np.where(home['home_team_goal'] > home['away_team_goal'], 3, np.where(home['home_team_goal'] == home['away_team_goal'], 1, 0)).copy()
        away['punten'] = np.where(away['home_team_goal'] < away['away_team_goal'], 3, np.where(away['home_team_goal'] == away['away_team_goal'], 1, 0)).copy()
        
        aantal_wed= len(home) + len(away)
        awaysum = away['punten'].sum()
        homesum = home['punten'].sum()
        total_punten = awaysum + homesum
        
        gem_punten.append(total_punten/aantal_wed)
    
    visualise_df['gemiddelde'] = pd.Series(gem_punten).values
    print(visualise_df)

In [6]:
teamseason('2008/2009')

C:\Users\tim4j\AppData\Local\Temp\ipykernel_13896\568924647.py:10: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  result10 = match_df.loc[(match_df['away_team_api_id'] == team_id)& (match_df['season'] == season)  | (match_df['home_team_api_id'] == team_id) & (match_df['season'] == season)]


ValueError: Cannot index with multidimensional key